In [2]:
import pickle as pkl
import spacy
import csv
import json
from copy import deepcopy
from tqdm import tqdm
import benepar
import re
from string import punctuation

In [3]:
from utils import merge_maximum_span
from utils import clean_sentence_brackets
from utils import process_nps_punctuation

In [4]:
sm_parser = spacy.load('en_core_web_sm')
berkeley_parser = spacy.load('en_core_web_md')
berkeley_parser.add_pipe("benepar", config={"model": "benepar_en3"})
trf_parser = spacy.load("en_core_web_trf")

In [5]:
# Load Parsed Corpus
sm_parser = spacy.load('en_core_web_sm')

# with open('parsed_corpus.pkl', 'rb') as f:
#     nps = pkl.load(f)
with open('parallel_data/parsed_corpus_all_zh.pkl', 'rb') as f:
    parsed_corpus = pkl.load(f)
inter_keys = parsed_corpus.keys()

In [17]:
for item in parsed_corpus:
    for scene in parsed_corpus[item]:
        for utt in scene:
            sentence = [item[0] for item in utt['berkeley_pron']]
            nps = process_nps_punctuation(sentence, process_nps_punctuation(sentence, utt['berkeley_noun_chunk']))
            text = " ".join(sentence)
            if "of" in sentence:
                print(text)
                print(nps)
                print()

I do n't know . I 've never reneged on a proffer of sperm before .
[('I', 0, 1), ('I', 5, 6), ('a proffer', 10, 12), ('sperm', 13, 14)]

If the height of a step is off by as little as two millimeters ...
[('the height', 1, 3), ('a step', 4, 6), ('as little as two millimeters', 9, 14)]

It 's true . I did a series of experiments when I was 12 . My father broke his clavicle .
[('It', 0, 1), ('I', 4, 5), ('a series', 6, 8), ('experiments', 9, 10), ('I', 11, 12), ('My father', 15, 17), ('his clavicle', 18, 20)]

That was the result of my work with lasers .
[('That', 0, 1), ('the result', 2, 4), ('my work', 5, 7), ('lasers', 8, 9)]

We 're gon na start season two of Battlestar Galactica .
[('We', 0, 1), ('season', 5, 6), ('Battlestar Galactica', 8, 10)]

Well , and that was wrong of us . We need to widen our circle .
[('that', 3, 4), ('us', 7, 8), ('We', 9, 10), ('our circle', 13, 15)]

Yes , and you 've never met one of them .
[('you', 3, 4), ('them', 9, 10)]

That 's the beauty of it .
[(

In [ ]:
for item in parsed_corpus:
    for scene in parsed_corpus[item]:
        for utt in scene:
            sentence = [item[0] for item in utt['berkeley_pron']]
            all_nps = process_nps_punctuation(sentence, process_nps_punctuation(sentence, utt['berkeley_noun_chunk']))
            if utt['berkeley_noun_chunk']!=all_nps:
                print(utt['berkeley_noun_chunk'])
                print(all_nps)
                print()

In [9]:
output = []
for epi_key in inter_keys:
    if epi_key != (1,9):
        continue
    for i in range(len(parsed_corpus[epi_key])):
        all_sentences = []
        all_query_spans = []

        scene_tag = parsed_corpus[epi_key][i]
        j = 0
        for utt in scene_tag:
            speaker = utt['speaker'].strip().strip("(").strip(")").strip().strip(".").strip().strip(":")
            speaker_tokens = [item.text for item in sm_parser(speaker)]
            sentence = utt['sentence']

            # print("Speaker:", speaker, speaker_tokens)
            # print("Sentences:", sentence)

            sentence_token = [item.text for item in sm_parser(sentence)]
            sm_nps = process_nps_punctuation(sentence_token, process_nps_punctuation(sentence_token, utt['sm_noun_chunk']))
            berkeley_nps = process_nps_punctuation(sentence_token, process_nps_punctuation(sentence_token, utt['berkeley_noun_chunk']))
            trf_nps = process_nps_punctuation(sentence_token, process_nps_punctuation(sentence_token, utt['trf_noun_chunk']))
            noun_phrase = merge_maximum_span(list(set(sm_nps) | set(berkeley_nps) | set(trf_nps)))
            temp_pron = []
            temp_pron.extend([(item[0], item[1], item[2]) for item in utt['sm_pron'] if item[3]=='PRON'])
            temp_pron.extend([(item[0], item[1], item[2]) for item in utt['berkeley_pron'] if item[3]=='PRON'])
            temp_pron.extend([(item[0], item[1], item[2]) for item in utt['trf_pron'] if item[3]=='PRON'])
            pron = merge_maximum_span(list(set(temp_pron)))
            # print('--')

            # print("Prons:", pron)
            # print("NPs", noun_phrase)
            # print()
            # print("SM:", utt['sm_noun_chunk'])
            # print()
            # print("Berkeley:", utt['berkeley_noun_chunk'])
            # print()
            # print("TRF:", utt['trf_noun_chunk'])

            # print(utt['sm_pron'])
            # print(utt['berkeley_pron'])
            # print(utt['trf_pron'])
            # print(noun_phrase)

            # print(utt['berkeley_noun_chunk'])
            for item in utt['berkeley_noun_chunk']:
                # if ("\"" in item[0]) or ("\'" in item[0]):
                # if ("\'" in item[0]):
                #     # print(sentence_token)
                #     print(item[0], item[1:])
                #     print(sentence_token[item[1]], sentence_token[item[2]-1])
                #     print('----')
                temp = list(deepcopy(item))
                if sentence_token[temp[1]] in punctuation:
                    # If the first token is quotation and there is not only one quotation, stop removing
                    if sentence_token[temp[1]]=="\"":
                        if " ".join(sentence_token[temp[1]: temp[2]]).count("\"")!=1:
                            continue
                    temp[1]-=1
                elif " ".join(sentence_token[temp[1]: temp[2]]).count("\"")%2==1:
                    if sentence_token[temp[2]]=="\"":
                        print(sentence_token)
                        print(sentence_token[temp[1]: temp[2]])
                        print('----')
            mention = list(set(noun_phrase)|set(pron))
            mention.sort(key=lambda x: x[1])

            all_sentences.append(speaker_tokens + [":"] + sentence_token)
            for span in mention:
                all_query_spans.append({
                        "sentenceIndex": j,
                        "startToken": span[1] + len(speaker_tokens) + 1,
                        "endToken": span[2] + len(speaker_tokens) + 1
                    })
            j+=1
            # print("=="*50)
        output.append({
            "sentences": all_sentences,
            "querySpans": all_query_spans,
            "candidateSpans": all_query_spans,
            "clickSpans": all_query_spans,
            })

['Have', 'fun', 'presenting', 'my', '"', 'lucky', 'hunch', '"']
['my', '"', 'lucky', 'hunch']
----


In [11]:
with open('temp_2.csv', "w", encoding="utf-8") as csv_fh:
        fieldnames = ['json_data']
        writer = csv.DictWriter(csv_fh, fieldnames, lineterminator='\n')
        writer.writeheader()
        for line in output:
            writer.writerow({'json_data': json.dumps(line)})

In [7]:
print(len(output))

8


In [81]:
print(3%2)

1


In [18]:
temp = deepcopy(output)
output = []
output.append(temp[4])
output.append(temp[6])
output.append(temp[2])

output.append(temp[1])
output.append(temp[3])
output.append(temp[5])
output.append(temp[7])
output.append(temp[0])

In [19]:
def combine_samples(instances):
    # Store Data
    sentences = []
    querySpans = []
    candidateSpans = []
    clickSpans = []
    sentence_offsets = [0]
    querySpans_offsets = [0]

    for instance in instances:
        offset = len(sentences)
        sentences.extend(instance['sentences'])
        for item in instance['querySpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            querySpans.append(token)
        for item in instance['candidateSpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            candidateSpans.append(token)
        for item in instance['clickSpans']:
            token = deepcopy(item)
            token['sentenceIndex'] = item['sentenceIndex'] + offset
            clickSpans.append(token)
        sentence_offsets.append(len(sentences))
        querySpans_offsets.append(len(querySpans))
    return {
    "sentences": sentences,
    "querySpans": querySpans,
    "candidateSpans": candidateSpans,
    "clickSpans": clickSpans,
    "sentence_offsets": sentence_offsets,
    "querySpans_offsets": querySpans_offsets,
    }

In [20]:
combined_output = []
combined_output.append(combine_samples(output[:3]))
combined_output.append(combine_samples(output[3:5]))

In [21]:
with open('combined_temp_2.csv', "w", encoding="utf-8") as csv_fh:
        fieldnames = ['json_data']
        writer = csv.DictWriter(csv_fh, fieldnames, lineterminator='\n')
        writer.writeheader()
        for line in combined_output:
            writer.writerow({'json_data': json.dumps(line)})

In [12]:
for item in combined_output:
    print(item['sentence_offsets'], len(item['sentence_offsets'])-1)

[0, 15, 47, 88] 3
[0, 36, 71] 2
